# Scaling XGBoost with Dask and Coiled

This notebook walks through training a distributed [XGBoost](https://xgboost.readthedocs.io/en/latest/) model locally on a small dataset using [Dask](https://dask.org/) and then using Dask and [Coiled](https://coiled.io/) to scale out to the cloud to run XGBoost on a larger-than-memory dataset.

## About the Dataset
We'll be using the Arcos dataset on Opioid sales as released by the Washington Post.
- Download the dataset here: https://www.washingtonpost.com/national/2019/07/18/how-download-use-dea-pain-pills-database/
- Washington Post Github repository here: https://github.com/wpinvestigative/arcos-api/
- Descriptions of columns here: https://github.com/wpinvestigative/arcos-api/blob/master/data/data_dictionary.csv

Includes:
- More than 178M transactions.
- Restricted to transactions where "Measure" is "Tab". This means the DOSAGE_UNIT field is the number of pills per tab.
- Restricted to oxycodone and hydrocodone


## TO DO
1. Get LocalCluster working (>> what's the point of including this?)
2. One-hot Encoding
3. Argumentation for XGBoost parameters
4. 

## 1. Importing Libraries

We'll start by importing all the libraries we'll need to run this notebook.

*Note how the the objects we import from **dask_ml** resemble the familiar sklearn API.*

In [1]:
import coiled
import dask.dataframe as  dd
from dask.distributed import Client, LocalCluster
from dask_ml.preprocessing import Categorizer, OneHotEncoder
from dask_ml.model_selection import train_test_split
import xgboost as xgb
from dask.distributed import performance_report

## 2. Local Distributed XGBoost Model using Dask

Next, let's instantiate a local version of the Dask distributed scheduler using the **LocalCluster** object. 

This object will handle parallelism for us on our local machine.

In [2]:
# local dask cluster
cluster = LocalCluster(n_workers=8)
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Status: running,Using processes: True
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 8,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:51845,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:51855,Total threads: 1
Dashboard: http://127.0.0.1:51856/status,Memory: 2.00 GiB
Nanny: tcp://127.0.0.1:51849,


In [2]:
# Create Coiled Cloud cluster
cluster = coiled.Cluster(
    name='xgboost',
    n_workers=20,
    worker_cpu=4,
    worker_memory='16GiB',
    software='coiled-examples/xgboost',
    shutdown_on_close=False,
    scheduler_options={'idle_timeout':'1hour'},
)

Output()

Found software environment build
Created FW rules: coiled-dask-rrpelgr71-36913-firewall
Created scheduler VM: coiled-dask-rrpelgr71-36913-scheduler (ip: ['34.201.64.247'])


In [5]:
# Connect Dask client to the Coiled cluster
client = Client(cluster)
client

<Client: 'tls://10.4.0.230:8786' processes=20 threads=40, memory=153.01 GiB>

In [5]:
# Download data from S3
data = dd.read_parquet(
    "s3://coiled-datasets/dea-opioid/arcos_washpost_comp.parquet", 
    compression="lz4",
    storage_options={"anon": True},
    columns=columns+categorical,
)

In [23]:
data_local = data.partitions[0:50]

In [24]:
# inspect the first 5 entries
data_local.head()

,QUANTITY,STRENGTH,CALC_BASE_WT_IN_GM,DOSAGE_UNIT,TRANSACTION_ID,REPORTER_BUS_ACT,REPORTER_NAME,REPORTER_CITY,REPORTER_STATE,REPORTER_ZIP,BUYER_BUS_ACT,BUYER_NAME,BUYER_CITY,BUYER_STATE,BUYER_ZIP,DRUG_NAME,UNIT,Product_Name,Revised_Company_Name
0,1.0,NaN,0.60540,100.0,64,DISTRIBUTOR,ACE SURGICAL SUPPLY CO INC,BROCKTON,MA,2301,PRACTITIONER,"TABRIZI, HAMID R DMD",MALDEN,MA,2148,HYDROCODONE,None,HYDROCODONE BIT/ACETA 10MG/500MG USP,Mallinckrodt
1,4.0,NaN,0.12108,40.0,52,DISTRIBUTOR,APOTHECA INC,PHOENIX,AZ,85006,RETAIL PHARMACY,APOTHECARY SHOP DEER VALLEY,PHOENIX,AZ,85085,HYDROCODONE,None,HYDROCODONE BITARTRATE & ACETA 5MG/,Apotheca Inc.
2,40.0,NaN,3.63240,1200.0,119,DISTRIBUTOR,APOTHECA INC,PHOENIX,AZ,85006,PRACTITIONER,"HOBBS, DOUGLAS DON, MD",GILBERT,AZ,85233,HYDROCODONE,None,HYDROCODONE BITARTRATE & ACETA 5MG/,Apotheca Inc.
3,20.0,NaN,2.72430,600.0,34,DISTRIBUTOR,APOTHECA INC,PHOENIX,AZ,85006,PRACTITIONER,"HOBBS, DOUGLAS DON, MD",GILBERT,AZ,85233,HYDROCODONE,None,HYDROCODONEBITARTRATE & ACETA 7.5MG,Apotheca Inc.
4,10.0,NaN,0.90810,300.0,19,DISTRIBUTOR,APOTHECA INC,PHOENIX,AZ,85006,PRACTITIONER,"HOBBS, DOUGLAS DON, MD",GILBERT,AZ,85233,HYDROCODONE,None,HYDROCODONE BITARTRATE & ACETA 5MG/,Apotheca Inc.


This is looking good.

Before we can start training our XGBoost model, however, we'll have to conduct two preprocessing steps:
1. Cast our categorical columns to the correct types (XGBoost only accepts float, integer and boolean dtypes)
2. Create our train and test splits

*Note: we're using the **dask_ml** library for this, which mimics the familiar scikit-learn API*

In [25]:
# Cast categorical columns to the correct type
ce = Categorizer(columns=categorical)
data_local = ce.fit_transform(data_local)
for col in categorical:
    data_local[col] = data_local[col].cat.codes

In [26]:
# Create the train-test split
X, y = data_local.iloc[:, :-1], data_local["DRUG_NAME"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, shuffle=True, random_state=2
)

Great, now we're all set to start training our XGBoost model.

First, we'll create the XGBoost DMatrix and set the model parameters.

In [27]:
# Create the XGBoost DMatrix

dtrain = xgb.dask.DaskDMatrix(client, X_train, y_train)

# Set parameters
params = {
    "max_depth": 8,
    "max_leaves": 2 ** 8,
    "gamma": 0.1,
    "eta": 0.1,
    "min_child_weight": 30,
    "objective": "binary:logistic",
    "grow_policy": "lossguide"
}


distributed.worker - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker.html#memtrim for more information. -- Unmanaged memory: 1.43 GiB -- Worker memory limit: 2.00 GiB
distributed.worker - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker.html#memtrim for more information. -- Unmanaged memory: 1.43 GiB -- Worker memory limit: 2.00 GiB
distributed.worker - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker.html#memtrim for more information. -- Unmanaged memory: 1.43 GiB -- Worker memory limit: 2.00 GiB
distributed.worker - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see 

Then let's go ahead and train the model.

In [28]:
%%time 
# train the model
output = xgb.dask.train(
    client, params, dtrain, num_boost_round=5,
    evals=[(dtrain, 'train')]
)

[13:38:33] task [xgboost.dask]:tcp://127.0.0.1:52260 got new rank 0
[13:38:33] task [xgboost.dask]:tcp://127.0.0.1:52256 got new rank 1
[13:38:33] task [xgboost.dask]:tcp://127.0.0.1:52252 got new rank 2
[13:38:33] task [xgboost.dask]:tcp://127.0.0.1:52240 got new rank 3
[13:38:33] task [xgboost.dask]:tcp://127.0.0.1:52242 got new rank 4
[13:38:33] task [xgboost.dask]:tcp://127.0.0.1:52239 got new rank 5
[13:38:33] task [xgboost.dask]:tcp://127.0.0.1:52241 got new rank 6
[13:38:33] task [xgboost.dask]:tcp://127.0.0.1:52248 got new rank 7
[13:38:33] WARNING: /Users/ktietz/demo/mc3/conda-bld/xgboost-split_1628682908089/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:38:33] WARNING: /Users/ktietz/demo/mc3/conda-bld/xgboost-split_1628682908089/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the def

CPU times: user 266 ms, sys: 30.8 ms, total: 297 ms
Wall time: 2.06 s


And see the results:

In [29]:
# 'booster' is the trained model
booster = output['booster']  

# 'history' is a dictionary containing evaluation metrics
history = output['history']  

## 3. Cloud-Based Distributed XGBoost using Dask and Coiled

Let's now expand this workflow to process the entire dataset (~200GB). We'll run almost exactly the same code except for **2 changes**:
1. We'll connect Dask to a Coiled cluster in the cloud, instead of to our local CPU cores,
2. We'll work with the entire dataset, instead of the first 50 partitions.

In the section below we've copied and pasted the cells from above so that you can run this notebook from top to bottom in one go. Alternatively, you could run the cell below (where we instantiate the Coiled Cluster) and then simply re-run the cells above -- making sure to adjust the cell that downloads the data as well, of course.

### Instantiate Coiled Cluster
Let's create our Coiled cluster in the cloud. We'll specify a cluster of 20 workers, with 4 CPU cores and 16GB of RAM each. That should allow the entire dataset to fit into the cluster's memory comfortably.

In [4]:
cluster = coiled.Cluster(
    name='xgboost',
    software='coiled-examples/xgboost',
    n_workers=20,
    scheduler_cpu=2,
    scheduler_memory='30Gib',
    shutdown_on_close=False,
    scheduler_options={'idle_timeout':'1hour'},
)

Output()

Using existing cluster: 'xgboost'

In [14]:
# Shut down the cluster
client.close()

## Import Data

In [2]:
columns = [
    "QUANTITY",
    "STRENGTH",
    "CALC_BASE_WT_IN_GM",
    "DOSAGE_UNIT",
]

In [3]:
categorical = [
    "TRANSACTION_ID",
    "REPORTER_BUS_ACT",
    "REPORTER_NAME",
    "REPORTER_CITY",
    "REPORTER_STATE",
    "REPORTER_ZIP",
    "BUYER_BUS_ACT",
    "BUYER_NAME",
    "BUYER_CITY",
    "BUYER_STATE",
    "BUYER_ZIP",
    "DRUG_NAME",
    "UNIT",
    "Product_Name",
    "Revised_Company_Name",
]

### Download the Data

In [6]:
# create Dask Dataframe with parquet data from S3
data = dd.read_parquet(
    "s3://coiled-datasets/dea-opioid/arcos_washpost_comp.parquet", 
    compression="lz4",
    storage_options={"anon": True},
    columns=columns+categorical,
)

data = data.partitions[0:1500]

### Preprocessing

In [46]:
cat = data[categorical]
cat = cat.drop('UNIT', axis=1)
cat.head()

,TRANSACTION_ID,REPORTER_BUS_ACT,REPORTER_NAME,REPORTER_CITY,REPORTER_STATE,REPORTER_ZIP,BUYER_BUS_ACT,BUYER_NAME,BUYER_CITY,BUYER_STATE,BUYER_ZIP,DRUG_NAME,Product_Name,Revised_Company_Name
0,64,DISTRIBUTOR,ACE SURGICAL SUPPLY CO INC,BROCKTON,MA,2301,PRACTITIONER,"TABRIZI, HAMID R DMD",MALDEN,MA,2148,HYDROCODONE,HYDROCODONE BIT/ACETA 10MG/500MG USP,Mallinckrodt
1,52,DISTRIBUTOR,APOTHECA INC,PHOENIX,AZ,85006,RETAIL PHARMACY,APOTHECARY SHOP DEER VALLEY,PHOENIX,AZ,85085,HYDROCODONE,HYDROCODONE BITARTRATE & ACETA 5MG/,Apotheca Inc.
2,119,DISTRIBUTOR,APOTHECA INC,PHOENIX,AZ,85006,PRACTITIONER,"HOBBS, DOUGLAS DON, MD",GILBERT,AZ,85233,HYDROCODONE,HYDROCODONE BITARTRATE & ACETA 5MG/,Apotheca Inc.
3,34,DISTRIBUTOR,APOTHECA INC,PHOENIX,AZ,85006,PRACTITIONER,"HOBBS, DOUGLAS DON, MD",GILBERT,AZ,85233,HYDROCODONE,HYDROCODONEBITARTRATE & ACETA 7.5MG,Apotheca Inc.
4,19,DISTRIBUTOR,APOTHECA INC,PHOENIX,AZ,85006,PRACTITIONER,"HOBBS, DOUGLAS DON, MD",GILBERT,AZ,85233,HYDROCODONE,HYDROCODONE BITARTRATE & ACETA 5MG/,Apotheca Inc.


In [47]:
enc = OneHotEncoder()
enc.fit(cat)

ValueError: All columns must be Categorical dtype when 'categories="auto"'.

In [9]:
# Cast categorical columns to the correct type
ce = Categorizer(columns=categorical)
data = ce.fit_transform(data)
for col in categorical:
    data[col] = data[col].cat.codes

In [10]:
data.dtypes

QUANTITY                float64
STRENGTH                float64
CALC_BASE_WT_IN_GM      float64
DOSAGE_UNIT             float64
TRANSACTION_ID            int32
REPORTER_BUS_ACT           int8
REPORTER_NAME             int16
REPORTER_CITY             int16
REPORTER_STATE             int8
REPORTER_ZIP              int16
BUYER_BUS_ACT              int8
BUYER_NAME                int32
BUYER_CITY                int16
BUYER_STATE                int8
BUYER_ZIP                 int16
DRUG_NAME                  int8
UNIT                       int8
Product_Name              int16
Revised_Company_Name       int8
dtype: object

In [11]:
data.head(10)

,QUANTITY,STRENGTH,CALC_BASE_WT_IN_GM,DOSAGE_UNIT,TRANSACTION_ID,REPORTER_BUS_ACT,REPORTER_NAME,REPORTER_CITY,REPORTER_STATE,REPORTER_ZIP,BUYER_BUS_ACT,BUYER_NAME,BUYER_CITY,BUYER_STATE,BUYER_ZIP,DRUG_NAME,UNIT,Product_Name,Revised_Company_Name
0,1.0,NaN,0.605400,100.0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0
1,4.0,NaN,0.121080,40.0,1,0,1,1,1,1,1,1,1,1,1,0,-1,1,1
2,40.0,NaN,3.632400,1200.0,2,0,1,1,1,1,0,2,2,1,2,0,-1,1,1
3,20.0,NaN,2.724300,600.0,3,0,1,1,1,1,0,2,2,1,2,0,-1,2,1
4,10.0,NaN,0.908100,300.0,4,0,1,1,1,1,0,2,2,1,2,0,-1,1,1
5,20.0,NaN,1.816200,600.0,5,0,1,1,1,1,0,2,2,1,2,0,-1,1,1
6,5.0,NaN,0.227025,75.0,6,0,1,1,1,1,0,3,3,1,3,0,-1,1,1
7,10.0,NaN,0.454050,150.0,7,0,1,1,1,1,0,3,3,1,3,0,-1,1,1
8,10.0,NaN,0.454050,150.0,8,0,1,1,1,1,0,3,3,1,3,0,-1,1,1
9,10.0,NaN,0.454050,150.0,9,0,1,1,1,1,0,3,3,1,3,0,-1,1,1


Categorical columns are now encoded.

NOTE: technically this should be **one-hot encoded** to avoid the ML model treating this as ORDINAL data.

In [12]:
# Create the train-test split
X, y = data.iloc[:, :-1], data["DRUG_NAME"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, shuffle=True, random_state=2
)

### Training Model

In [ ]:
# Create the XGBoost DMatrix
dtrain = xgb.dask.DaskDMatrix(client, X_train, y_train)

# Set model parameters
params = {
    "max_depth": 8,
    "max_leaves": 2 ** 8,
    "gamma": 0.1,
    "eta": 0.1,
    "min_child_weight": 30,
    "objective": "binary:logistic",
    "grow_policy": "lossguide"
}


In [9]:
%%time 
# train the model (and generate a Dask performance report)
with performance_report(filename="xgboost-training.html"):
    output = xgb.dask.train(
        client, params, dtrain, num_boost_round=5,
        evals=[(dtrain, 'train')]
    )

CPU times: user 749 ms, sys: 562 ms, total: 1.31 s
Wall time: 24.7 s


distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError


In [ ]:
# 'booster' is the trained model
booster = output['booster']  

# 'history' is a dictionary containing evaluation metrics
history = output['history']  

### Shutting down the cluster

In [29]:
# Shut down the cluster
client.close()

## 4. Recap

In this notebook, we:
- trained a distributed XGBoost model on a portion of the XXX dataset using all of the cores of our machine in parallel by instantiating a Dask LocalCluster,
- expanded the distributed XGBoost model to train on the entire dataset using a Coiled Cluster of XX machines and XX total memory in the cloud.

We’d love to see you apply distributed XGBoost to a dataset that’s meaningful to you. If you’d like to try, swap your own dataset into this notebook and see how well it does! 

Let us know how you get on in our [Coiled Community Slack channel](https://join.slack.com/t/coiled-users/shared_invite/zt-hx1fnr7k-In~Q8ui3XkQfvQon0yN5WQ) or by tweeting at us.

## Preprocessing Internal Only

In [18]:
data.DRUG_NAME.value_counts().compute()

KeyboardInterrupt: 

In [7]:
# inspect the first 5 entries
data.head()

,QUANTITY,STRENGTH,CALC_BASE_WT_IN_GM,DOSAGE_UNIT,TRANSACTION_ID,REPORTER_BUS_ACT,REPORTER_NAME,REPORTER_CITY,REPORTER_STATE,REPORTER_ZIP,BUYER_BUS_ACT,BUYER_NAME,BUYER_CITY,BUYER_STATE,BUYER_ZIP,DRUG_NAME,UNIT,Product_Name,Revised_Company_Name
0,1.0,NaN,0.60540,100.0,64,DISTRIBUTOR,ACE SURGICAL SUPPLY CO INC,BROCKTON,MA,2301,PRACTITIONER,"TABRIZI, HAMID R DMD",MALDEN,MA,2148,HYDROCODONE,None,HYDROCODONE BIT/ACETA 10MG/500MG USP,Mallinckrodt
1,4.0,NaN,0.12108,40.0,52,DISTRIBUTOR,APOTHECA INC,PHOENIX,AZ,85006,RETAIL PHARMACY,APOTHECARY SHOP DEER VALLEY,PHOENIX,AZ,85085,HYDROCODONE,None,HYDROCODONE BITARTRATE & ACETA 5MG/,Apotheca Inc.
2,40.0,NaN,3.63240,1200.0,119,DISTRIBUTOR,APOTHECA INC,PHOENIX,AZ,85006,PRACTITIONER,"HOBBS, DOUGLAS DON, MD",GILBERT,AZ,85233,HYDROCODONE,None,HYDROCODONE BITARTRATE & ACETA 5MG/,Apotheca Inc.
3,20.0,NaN,2.72430,600.0,34,DISTRIBUTOR,APOTHECA INC,PHOENIX,AZ,85006,PRACTITIONER,"HOBBS, DOUGLAS DON, MD",GILBERT,AZ,85233,HYDROCODONE,None,HYDROCODONEBITARTRATE & ACETA 7.5MG,Apotheca Inc.
4,10.0,NaN,0.90810,300.0,19,DISTRIBUTOR,APOTHECA INC,PHOENIX,AZ,85006,PRACTITIONER,"HOBBS, DOUGLAS DON, MD",GILBERT,AZ,85233,HYDROCODONE,None,HYDROCODONE BITARTRATE & ACETA 5MG/,Apotheca Inc.


In [8]:
data.columns

Index(['QUANTITY', 'STRENGTH', 'CALC_BASE_WT_IN_GM', 'DOSAGE_UNIT',
       'TRANSACTION_ID', 'REPORTER_BUS_ACT', 'REPORTER_NAME', 'REPORTER_CITY',
       'REPORTER_STATE', 'REPORTER_ZIP', 'BUYER_BUS_ACT', 'BUYER_NAME',
       'BUYER_CITY', 'BUYER_STATE', 'BUYER_ZIP', 'DRUG_NAME', 'UNIT',
       'Product_Name', 'Revised_Company_Name'],
      dtype='object')

### Process from here:
1. Get categorical columns
2. Preprocess those

In [7]:
data.REPORTER_BUS_ACT.value_counts().compute()

DISTRIBUTOR              178082880
MANUFACTURER                514415
REVERSE DISTRIB                730
CHEMICAL MANUFACTURER            1
Name: REPORTER_BUS_ACT, dtype: int64

In [31]:
data.REPORTER_NAME.value_counts().compute()

MCKESSON CORPORATION            36970763
CARDINAL HEALTH                 25362775
WALGREEN CO                     23047615
AMERISOURCEBERGEN DRUG CORP     15278813
WAL-MART PHARM WAREHOUSE #1      5682151
                                  ...   
BROOKSHIRE GROCERY COMPANY             1
OCEAN MEDICAL                          1
SAVOY MEDICAL SUPPLY COMPANY           1
MCKESSON SPECIALTY LOGISTICS           1
BOCA PHARMACAL LLC                     1
Name: REPORTER_NAME, Length: 494, dtype: int64

In [12]:
data.REPORTER_CITY.value_counts().compute()

ROGERS                9888365
PERRYSBURG            6102999
WOODLAND              5030005
JUPITER               4730129
KNOXVILLE             4439162
                       ...   
CUMBERLAND                  1
HILTON HEAD ISLAND          1
CORAL SPRINGS               1
CONWAY                      1
LINCOLN                     1
Name: REPORTER_CITY, Length: 418, dtype: int64

In [14]:
data.REPORTER_STATE.value_counts().head(10)

CA    19715990
OH    12204507
FL    12174208
AR    11156357
TX     9234960
IL     7994830
GA     7677366
TN     7501719
NY     7333245
IN     7261726
Name: REPORTER_STATE, dtype: int64

In [15]:
data.BUYER_BUS_ACT.value_counts().compute()

CHAIN PHARMACY         116383479
RETAIL PHARMACY         61053356
PRACTITIONER             1032103
PRACTITIONER-DW/30         59670
PRACTITIONER-DW/100        53702
PRACTITIONER-DW/275        15716
Name: BUYER_BUS_ACT, dtype: int64

In [17]:
data.QUANTITY.value_counts().compute()

1.0        97341738
2.0        35231179
3.0        15903312
4.0         8747752
5.0         5515753
             ...   
1660.0            1
679.0             1
1633.0            1
684.0             1
88880.0           1
Name: QUANTITY, Length: 1159, dtype: int64

In [22]:
data[['QUANTITY', 'UNIT', 'STRENGTH', 'DOSAGE_UNIT']].head()

,QUANTITY,UNIT,STRENGTH,DOSAGE_UNIT
0,1.0,None,NaN,100.0
1,4.0,None,NaN,40.0
2,40.0,None,NaN,1200.0
3,20.0,None,NaN,600.0
4,10.0,None,NaN,300.0


In [19]:
data.UNIT.value_counts().compute()

D    55
2    53
1    48
Name: UNIT, dtype: int64

May be worth including only entries for which UNIT = None. In that case we know for sure we're measuring the 

In [18]:
data.STRENGTH.value_counts().compute()

0.0       42992312
1000.0     1116037
800.0         2481
600.0         2140
100.0         1284
            ...   
896.0            1
910.0            1
915.0            1
943.0            1
8000.0           1
Name: STRENGTH, Length: 350, dtype: int64

STRENGTH is "One of three values: "(1) the purity of a bulk rawmaterial (2) the fractional portion of a standard NDC package size or (3) the percentage by which a package exceeds a standard NDC package size."

In [24]:
data.Measure.value_counts().compute()

TAB    178598026
Name: Measure, dtype: int64

In [25]:
data.DOSAGE_UNIT.value_counts().compute()

100.0        62033577
500.0        38699261
200.0        27531578
300.0        13225804
1000.0       10362267
               ...   
10332.0             1
10192.0             1
10164.0             1
9950.0              1
3115000.0           1
Name: DOSAGE_UNIT, Length: 3114, dtype: int64

DOSAGE_UNIT is DEA-calculated field indicating number of pills for transactions where Measure is Tab (all transactions for this dataset).

In [26]:
data.CALC_BASE_WT_IN_GM.value_counts().compute()

4.540500e-01    16392694
6.054000e-01    16020445
3.027000e+00    13221388
1.513500e+00    12902288
2.270250e+00     9890539
                  ...   
4.209043e+01           1
4.249410e+01           1
4.258989e+01           1
4.290772e+01           1
3.405375e+07           1
Name: CALC_BASE_WT_IN_GM, Length: 5603, dtype: int64

CALC_BASE_WT_IN_GM is a DEA-added field indicating active weight of the drug in the transaction, in grams

In [27]:
data.Product_Name.value_counts().compute()

HYDROCODONE BIT. 10MG/ACETAMINOPHEN     12564663
HYDROCODONE BIT/ACETAMINOPHEN 5MG/50    10722861
OXYCODONE HCL/ACETAMINOPHEN 5MG/325M     7547523
HYDROCODONE BIT 5MG/ACETAMINOPHEN 50     6587821
HYDROCODONE BIT/ACETA 7.5MG/500MG US     6000824
                                          ...   
VICODIN HYDROCODO.BIT.5MG/AC                   1
OXYCOD 5MG HCL/ACET 325MG TAB USP              1
HYDROCODO.BIT 7.5&AC USP TAB                   1
HYDROCODONE BIT/ACETA 7.5MG/750MG US           1
VICODIN ES TAB 7.5MG HYDROCODO.BIT/7           1
Name: Product_Name, Length: 1109, dtype: int64

In [30]:
data.Revised_Company_Name.value_counts().compute()

Mallinckrodt                                            65380874
Allergan, Inc.                                          52458781
Endo Pharmaceuticals, Inc.                              26366516
Purdue Pharma LP                                        13934502
Amneal Pharmaceuticals, Inc.                             6686972
                                                          ...   
Pharmaceutical Manufacturing Research Services, Inc.         110
Coupler Enterprises                                           73
Prepackage Specialists                                        52
Rx Pak Division of McKesson Corporation                        5
United Research Laboratories, Inc.                             2
Name: Revised_Company_Name, Length: 85, dtype: int64

In [28]:
data.Ingredient_Name.value_counts().compute()

HYDROCODONE BITARTRATE HEMIPENTAHYDRATE                109664762
OXYCODONE HYDROCHLORIDE                                 68933034
POLY ETHYLENE GLYCOL OXYCODOL (MPEG 6-ALPHA-OXYCODO          230
Name: Ingredient_Name, dtype: int64

In [29]:
data.dos_str.value_counts().compute()

10.0000     57865691
5.0000      47355052
7.5000      45693133
30.0000      5948201
20.0000      5263221
15.0000      5105884
40.0000      4904995
80.0000      3869498
60.0000       955627
2.5000        818028
5.3500        328988
4.8355        174280
45.0000          560
120.0000         172
100.0000         113
200.0000          95
400.0000          22
50.0000            5
Name: dos_str, dtype: int64

dos_str is the strength of the dose in milligrams